In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
from sklearn.metrics.pairwise import linear_kernel

In [2]:
books = pd.read_csv("../data/cleaned_books.csv")

In [3]:
books["mod_title"] = books["Title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)
books["mod_title"] = books["mod_title"].str.lower()
books["mod_title"] = books["mod_title"].str.replace("\s+", " ", regex=True)
books = books[books["mod_title"].str.len() > 0]

In [4]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143505 entries, 0 to 143504
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Title        143505 non-null  object
 1   image        136593 non-null  object
 2   authors      141755 non-null  object
 3   categories   137888 non-null  object
 4   publisher    125147 non-null  object
 5   description  143505 non-null  object
 6   mod_title    143505 non-null  object
dtypes: object(7)
memory usage: 7.7+ MB


In [5]:
books.head()

,Title,image,authors,categories,publisher,description,mod_title
0,Dr. Seuss: American Icon,http://books.google.com/books/content?id=IjvHQ...,['Philip Nel'],['Biography & Autobiography'],A&C Black,Philip Nel takes a fascinating look into the k...,dr seuss american icon
1,Wonderful Worship in Smaller Churches,http://books.google.com/books/content?id=2tsDA...,['David R. Ray'],['Religion'],NaN,This resource includes twelve principles in un...,wonderful worship in smaller churches
2,Whispers of the Wicked Saints,http://books.google.com/books/content?id=aRSIg...,['Veronica Haddon'],['Fiction'],iUniverse,Julia Thomas finds her life spinning out of co...,whispers of the wicked saints
3,The Church of Christ: A Biblical Ecclesiology ...,http://books.google.com/books/content?id=kVqRa...,['Everett Ferguson'],['Religion'],Wm. B. Eerdmans Publishing,In The Church of Christ: A Biblical Ecclesiolo...,the church of christ a biblical ecclesiology f...
4,Saint Hyacinth of Poland,http://books.google.com/books/content?id=lmLqA...,['Mary Fabyan Windeatt'],['Biography & Autobiography'],Tan Books & Pub,The story for children 10 and up of St. Hyacin...,saint hyacinth of poland


In [7]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(books['mod_title'])

In [8]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Preview</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

def search(query,vectorizer,tfidf):
    processed = re.sub("[^a-zA-Z0-9 ]", "", query.lower())
    query_vec = vectorizer.transform([query])
    cosine_similarities = linear_kernel(query_vec, tfidf).flatten()
    book_scores = list(enumerate(cosine_similarities))
    book_scores = sorted(book_scores, key=lambda x: x[1], reverse=True)
    top_books = book_scores[:10]  # Adjust the number of top results as needed
    top_book_indices = [i[0] for i in top_books]
    results = books.loc[top_book_indices,['Title','authors','description','image','publisher','categories']]
    
    return results.head(10).style.format({'previewLink': make_clickable, 'image': show_image})

In [29]:
search("mein kamp", vectorizer,tfidf)

,Title,authors,description,image,publisher,categories
17227,Mein Kampf,['Adolf Hitler'],"Hitler's infamous political tract was first published in 1925-26 and has been widely translated since. This edition contains a detailed introduction which analyses Hitler's background, his ideology and his ruthless understanding of political power.",,nan,['Germany']
126065,Mein Kampf: My Struggle,['Adolf Hitler'],"‘MEIN KAMPF’ is the autobiography of Adolf Hitler gives detailed insight into the mission and vision of Adolf Hitler that shook the world. This book is the merger of two volumes. The first volume of MEIN KAMPF’ was written while the author was imprisioned in a Bavarian fortress. The book deals with events which brought the author into this blight. It was the hour of Germany’s deepest humiliation, when Napolean has dismembered the old German Empire and French soldiers occupied almost the whole of Germony. The books narrates how Hitler was arrested with several of his comrades and imprisoned in the fortress of Landsberg on the river Lech. During this period only the author wrote the first volume of MEIN KAMPF. The Second volume of MEIN KAMPF was written after release of Hitler from prison and it was published after the French had left the Ruhr, the tramp of the invading armies still echoed in German ears and the terrible ravages had plunged the country into a state of social and economic Chaos. The beauty of the book is, MEIN KAMPF is an historical document which bears the emprint of its own time. Moreover, Hitler has declared that his acts and ‘public statements’ constitute a partial revision of his book and are to be taken as such. Also, the author has translated Hitler’s ideal, the Volkischer Staat, as the People’s State. The author has tried his best making German Vocabulary easy to understand. You will never be satisfied until go through the whole book. A must read book, which is one of the most widely circulated and read books worldwide.",,Diamond Pocket Books Pvt Ltd,['Biography & Autobiography']
46152,"Hitler's ""Mein Kampf"": An Analysis",['Adolf Hitler'],"‘MEIN KAMPF’ is the autobiography of Adolf Hitler gives detailed insight into the mission and vision of Adolf Hitler that shook the world. This book is the merger of two volumes. The first volume of MEIN KAMPF’ was written while the author was imprisioned in a Bavarian fortress. The book deals with events which brought the author into this blight. It was the hour of Germany’s deepest humiliation, when Napolean has dismembered the old German Empire and French soldiers occupied almost the whole of Germony. The books narrates how Hitler was arrested with several of his comrades and imprisoned in the fortress of Landsberg on the river Lech. During this period only the author wrote the first volume of MEIN KAMPF. The Second volume of MEIN KAMPF was written after release of Hitler from prison and it was published after the French had left the Ruhr, the tramp of the invading armies still echoed in German ears and the terrible ravages had plunged the country into a state of social and economic Chaos. The beauty of the book is, MEIN KAMPF is an historical document which bears the emprint of its own time. Moreover, Hitler has declared that his acts and ‘public statements’ constitute a partial revision of his book and are to be taken as such. Also, the author has translated Hitler’s ideal, the Volkischer Staat, as the People’s State. The author has tried his best making German Vocabulary easy to understand. You will never be satisfied until go through the whole book. A must read book, which is one of the most widely circulated and read books worldwide.",,Diamond Pocket Books Pvt Ltd,['Biography & Autobiography']
0,Dr. Seuss: American Icon,['Philip Nel'],"Philip Nel takes a fascinating look into the key aspects of Seuss's career - his poetry, politics, art, marketing, and place in the popular imagination."" ""Nel argues convincingly that Dr. Seuss is one of the most influential poet

In [ ]:
liked_books = ['57079','10226','51004','103682','209361','144928']

In [ ]:
# books.loc[15389,'Title']
pd.DataFrame(books.loc[15389]).transpose().style.format({'image':show_image})

,Title,image,authors,categories,publisher,description,mod_title
15389,The Return of the Indian,,['Lynne Reid Banks'],['Juvenile Fiction'],Yearling Books,"A year after he sends his Indian friend, Little Bear, back into the magic cupboard, Omri decides to bring him back only to find that he is close to death and in need of help.",the return of the indian
